In [3]:
from numpy import loadtxt, pi
path = loadtxt(r"C:\Users\Srinivas\Documents\GitHub\New_world\robot_trajectory.txt")
poses = []
for i in range(len(path)):
    if path[i][3]!=0:
        if path[i][3] == 1:
            poses.append([path[i][0],path[i][1],path[i-1][2],pi])
        elif path[i][3] == 3:
            poses.append([path[i][0],path[i][1],path[i-1][2],0])
        elif path[i][3] == 2:
            poses.append([path[i][0],path[i][1],path[i-1][2],3*pi/2])
        elif path[i][3] == 4:
            poses.append([path[i][0],path[i][1],path[i-1][2],pi/2])

lposes = len(poses)

In [4]:
from math import tan, atan, sqrt, cos, sin
from numpy import eye, diag, array, exp, ceil, floor, dot
from numpy.linalg import inv, det
from random import gauss, randint, uniform
import copy
import sys

class Particle:
    def __init__(self,pose):
        self.pose = pose
        self.landmark_poses = []
        self.landmark_covariances = []
        self.hist_landmark = ['map',['0'],['pi/2'],['pi'],['3pi/2']]
        
    def number_of_landmarks(self):
        return len(self.landmark_poses)
    
    @staticmethod
    def motion(pose, control, WB, dt):  
        angle = (control[1]+pose[2])%(2*pi)
        return ([x+y for x,y in zip(pose,[control[0]*dt*cos(angle),
                                          control[0]*dt*sin(angle),
                                          control[0]*dt*sin(control[1])/WB])])
    
    def move(self,control,WB,dt,pose):
        self.pose = pose
        
    def expected_measurement(self,pose,landmark):     
        return array([landmark[0],landmark[1],tan(landmark[2]-pose[2])])      
    
    def measurement_correspondence(self,pose,measurement,number_of_landmarks,Qt_measurement_covariance):
        likelihoods = []
        for i in range(number_of_landmarks):
            likelihoods.append(self.landmark_correspondence(measurement,i,Qt_measurement_covariance))
        return likelihoods
    
    @staticmethod
    def dh_dlandmark(pose,landmark):
        return array([[1,0,0],[0,1,0],[0,0,-1/(cos(pose[2]-landmark[2])**2)]]) # H - Direct odometry measurement and collision
    
    def H_and_Ql_from_measurement(self,landmark_number,Qt_measurement_covariance):
        
        H = self.dh_dlandmark(self.pose,self.landmark_poses[landmark_number])
        Ql = dot(dot(H,self.landmark_covariances[landmark_number]),H.T) + Qt_measurement_covariance
        
        return (H, Ql)
        
    def landmark_correspondence_likelihood(self,measurement,landmark_number,Qt_measurement_covariance):
        # For a given measurment and a landmark number, it returns a suitable likelihood value of the correspondence

        landmark = self.landmark_poses[landmark_number]
        phi = landmark[2]
        theta = self.pose[2]
        
        # Prune the landmark which are not in the robot's frame of collision
        if phi == pi/2 * floor(theta/(pi/2)) or phi == pi/2 * ceil(theta/(pi/2)): 
            zhat = self.expected_measurement(self.pose,landmark)
            H, Ql = self.H_and_Ql_from_measurement(landmark_number,Qt_measurement_covariance)
            dz = measurement - zhat
            # Compute likelihood
            sqrtdetQl = sqrt(det(Ql))
            normal = 1 / (2*pi*sqrtdetQl)
            l = normal * exp(-0.5*dot(dot(dz.T,inv(Ql)),dz))            
        else: 
            l = 0
        return l
    
    def initialize_new_landmark(self,measurement,Qt_measurement_covariance):

        self.landmark_poses.append([self.pose[0],self.pose[1],self.pose[2]+atan(measurement[2])]) # Orientation update as FSM
        # We have to modify the orientation of the landmark to multiples of 90 degrees
        Hinv = array([[1,0,0],[0,1,0],[0,0,1/(1+measurement[2]**2)]])
        Sigma = dot(dot(Hinv,Qt_measurement_covariance),Hinv.T)
        # Remember to add the covariance of the particle at that moment
        
        self.landmark_covariances.append(Sigma)
        
    def update_landmark(self,measurement,landmark_number,Qt_measurement_covariance):
        
        mu = self.landmark_poses[landmark_number]
        Sigma = self.landmark_covariances[landmark_number]
        H, Ql = self.H_and_Ql_from_measurement(landmark_number,Qt_measurement_covariance)
        K = dot(dot(Sigma,H.T),inv(Ql))
        
        mu = mu + dot(K,(measurement - self.expected_measurement(self.pose,mu)))
        Sigma = dot((eye(3) - K*H),Sigma)
        
        return (mu,Sigma)
    
    def update_particle(self,measurement,number_of_landmarks,minimum_correspondence_likelihood,Qt_measurement_covariance):
        likelihoods = []
        for i in range(number_of_landmarks):
            likelihoods.append(self.landmark_correspondence_likelihood(measurement,i,Qt_measurement_covariance))

        if not likelihoods or max(likelihoods) < minimum_correspondence_likelihood:
            self.initialize_new_landmark(measurement,Qt_measurement_covariance)
            return minimum_correspondence_likelihood
        
        else:
            lmax = max(likelihoods)
            lmax_index = likelihoods.index(lmax)
            mu,Sigma = self.update_landmark(measurement,lmax_index,Qt_measurement_covariance)
            self.landmark_poses[lmax_index] = mu
            self.landmark_covariances[lmax_index] = Sigma
            
            return lmax

class FastSLAM:
    def __init__(self,initial_particles,robot_width,minimum_correspondence_likelihood,measurement_stddev,
                 x_stddev,y_stddev,control_speed_factor,control_head_factor, sample_time):
        # Particles
        self.particles = initial_particles
        
        # Constants
        self.robot_width = robot_width
        self.minimum_correspondence_likelihood = minimum_correspondence_likelihood
        self.xstddev = xstddev
        self.ystddev = ystddev
        self.measurement_stddev = measurement_stddev
        self.control_speed_factor = control_speed_factor
        self.control_head_factor = control_head_factor
        self.dt = sample_time
        self.WB = robot_width
        
    def predict(self,pose,control):
        # Prediction step of FastSLAM

        speed, head = control
        speed_std = self.control_speed_factor * sqrt(speed) # To be modified
        head_std  = self.control_head_factor * sqrt(359 - head) # To be modified; mirror image of the speed deviation
        
        for p in self.particles:
            speed = gauss(speed,speed_std)
            head = gauss(head,head_std)
            p.move([speed,head],self.WB,self.dt,pose)
    
    def update_and_compute_weights(self,measurement):

        Qt_measurement_covariance = diag([self.xstddev ** 2, self.ystddev ** 2,self.measurement_stddev ** 2]) 
        weights = []
        for p in self.particles:
            number_of_landmarks = p.number_of_landmarks()
            weight = p.update_particle(measurement,number_of_landmarks,self.minimum_correspondence_likelihood,
                                            Qt_measurement_covariance)
            weights.append(weight)
            
        return weights
    
    def resample(self,weights):
        
        new_particles = []
        max_weight = max(weights)
        index = randint(0, len(self.particles)-1)
        offset = 0.0
        for i in xrange(len(self.particles)):
            offset += uniform(0, 2.0 * max_weight)
            while offset > weights[index]:
                offset -= weights[index]
                index = (index + 1) % len(weights)
            new_particles.append(copy.deepcopy(self.particles[index]))
            
        return new_particles
    
    def correct(self,measurement):

        weights = self.update_and_compute_weights(measurement)
        self.particles = self.resample(weights)
        
        return weights
        
if __name__ == '__main__':
    
    robot_width = 2
    sample_time = 0.01
    
    minimum_correspondence_likelihood = 1e-3
    xstddev = 0.001
    ystddev = 0.001
    measurement_stddev = 0.001
    
    control_speed_factor = 0.01
    control_head_factor = 0.01
    number_of_particles = 10
    
    start_state = poses[0][0:3]
    initial_particles = [copy.copy(Particle(start_state))
                         for _ in xrange(number_of_particles)]

    fs = FastSLAM(initial_particles,robot_width,minimum_correspondence_likelihood,measurement_stddev,xstddev,ystddev,
                 control_speed_factor,control_head_factor, sample_time)
    
    for i in xrange(lposes-1):
        # Correction step
        print "Iteration",i
        '''Use of entire trajectory for the correction step (Rao-Blackwellization)''' 
        w = fs.correct(array([poses[i][0],poses[i][1],tan(poses[i][3]-poses[i][2])])) # Odometry and collision values
        #Prediction
        fs.predict(poses[i+1][0:3],[100,poses[i+1][2]])

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Iteration 20
Iteration 21
Iteration 22
Iteration 23
Iteration 24
Iteration 25
Iteration 26
Iteration 27
Iteration 28
Iteration 29
Iteration 30
Iteration 31
Iteration 32
Iteration 33
Iteration 34
Iteration 35
Iteration 36
Iteration 37
Iteration 38
Iteration 39
Iteration 40
Iteration 41
Iteration 42
Iteration 43
Iteration 44
Iteration 45
Iteration 46
Iteration 47
Iteration 48
Iteration 49
Iteration 50
Iteration 51
Iteration 52
Iteration 53
Iteration 54
Iteration 55
Iteration 56
Iteration 57
Iteration 58
Iteration 59
Iteration 60
Iteration 61
Iteration 62
Iteration 63
Iteration 64
Iteration 65
Iteration 66
Iteration 67
Iteration 68
Iteration 69
Iteration 70
Iteration 71
Iteration 72
Iteration 73
Iteration 74
Iteration 75
Iteration 76
Iteration

In [5]:
import matplotlib.pyplot as plt
from time import sleep
from IPython import display

xl = []
yl = []
xt = []
yt = []

for i in xrange(len(fs.particles[0].landmark_poses)):
    xl.append(fs.particles[0].landmark_poses[i][0])
    yl.append(fs.particles[0].landmark_poses[i][1])
    xt.append(poses[i][0])
    yt.append(poses[i][1])

plt.plot(xl,yl,'o',color='b')
plt.plot(xt,yt,'o',color='r')
plt.show()
print "xl is",xl
print "yl is",yl
print "xt is",xt
print "yt is",yt

xl is [111.5, 22.0, 111.5, 22.0, 72.0, 236.0, 141.5, 341.0, 360.5, 266.0, 360.5, 266.0, 421.0, 480.0, 390.5, 480.0, 390.5, 480.0, 439.0, 390.5, 480.0, 390.5, 480.0, 339.0, 266.0, 360.5, 266.0, 360.5, 259.0, 141.5, 236.0, 141.5, 236.0, 179.0, 22.0, 111.5, 22.0, 111.5, 79.0, 22.0, 111.5, 22.0, 111.5, 22.0, 65.0, 236.0, 141.5, 334.0, 360.5, 266.0, 360.5, 266.0, 414.0, 480.0, 390.5, 480.0, 390.5, 480.0, 446.0, 390.5, 480.0, 390.5, 480.0, 346.0, 266.0, 360.5, 266.0, 360.5, 266.0, 141.5, 236.0, 141.5, 236.0, 186.0, 22.0, 111.5, 22.0, 111.5, 86.0, 22.0, 111.5, 22.0, 111.5, 22.0, 58.0, 236.0, 141.5, 327.0, 360.5, 266.0, 360.5, 266.0, 407.0, 480.0, 390.5, 480.0, 390.5]
yl is [161.5, 251.0, 340.5, 430.0, 480.0, 316.0, 221.5, 22.0, 41.5, 136.0, 230.5, 325.0, 480.0, 421.0, 331.5, 242.0, 152.5, 63.0, 22.0, 70.5, 160.0, 249.5, 339.0, 480.0, 407.0, 312.5, 218.0, 123.5, 22.0, 139.5, 234.0, 328.5, 423.0, 480.0, 323.0, 233.5, 144.0, 54.5, 22.0, 79.0, 168.5, 258.0, 347.5, 437.0, 480.0, 309.0, 214.5, 22.0

In [ ]:
print poses